In [41]:
#Mine
# Client ID: b738ddf1b6a94c31beab033d47f9a0df
# Client secret: 026caf874e7249d6a9b9b97dde921f85
#amaan
# Client ID: 2387f85230364ac3855f8a42afab007a
# Client Secret: ce13d37312ed425ab4146f522ed7891d
#Vashes
# Client ID:897df63598cd4b17bd8457f3413c4110
# Client secret:d86ed3485f984510989939cdcec9e366
#Nithu
# Client id: 9bf29085a2f94ea1a40e718cbc222bb7
# Client secret: d90389cc6c1f4cfaa6ec34ee567c0f5c
#Arti's
# Client ID:3b45ee233b334a9ebf11157eca39fda8
# Client secret:f28a3383f9714e86bd5f8820d08b62fe
# Anands:
# Client ID:3dfd571c547b4f1292dda2df6d86d942
# Client secret:b7487f767f174ee195e503d7bad7c3af

In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from datetime import datetime, timedelta
import os
import json

# Obtain Access Token

In [3]:
client_id = '2387f85230364ac3855f8a42afab007a'
client_secret = 'ce13d37312ed425ab4146f522ed7891d'
token_url = 'https://accounts.spotify.com/api/token'
body_params = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=body_params, auth=(client_id, client_secret))
print(response.json())


{'access_token': 'BQD9aG-RnoKAvt6-A7pZmYJwpRMTt-Uj1MZ21PX2mVdGnDBuGmsbgiUg8i43gUD9R1336GwjyF57eQhgHIPdjoYbf5bCGQFzDshFW1SAk7tVSdvaD_I', 'token_type': 'Bearer', 'expires_in': 3600}


# List the genres

In [ ]:
access_token = 'BQCf7vcL8ol4hRwrnEDFtgj3rwZQgIqugOxXyxrCpAQdQLDx4rtHwUofQ5vgT3N88m7521Wh4dZRgZtlCLonI7b629PA7tYz3jxa8vIWDGUjt5GU0Fk'
url = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'
headers = {
    'Authorization': f'Bearer {access_token}',
}
response = requests.get(url, headers=headers)
print(response.json())

{'genres': ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep'

# Obtain the album and song info, no audio features

In [ ]:

# List of genres to process
genres = ['pop', 'hip-hop', 'rock', 'electronic', 'r-n-b', 'jazz', 'country',
          'classical', 'reggae', 'alternative', 'indie', 'metal', 'funk', 'blues', 'soul']

access_token = 'BQCf7vcL8ol4hRwrnEDFtgj3rwZQgIqugOxXyxrCpAQdQLDx4rtHwUofQ5vgT3N88m7521Wh4dZRgZtlCLonI7b629PA7tYz3jxa8vIWDGUjt5GU0Fk'

if not os.path.exists('previews'):
    os.makedirs('previews')
if not os.path.exists('data'):
    os.makedirs('data')

def get_access_token(client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    response = requests.post(url, data={'grant_type': 'client_credentials'}, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print("Failed to refresh token")
        return None

def make_request(url, token):
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 401:  
        token = get_access_token(client_id, client_secret)
        if token:
            headers = {'Authorization': f'Bearer {token}'}
            response = requests.get(url, headers=headers)  
        else:
            print("Error: Unable to fetch new access token.")
            exit()
    if response.status_code != 200:
        return None, token
    return response.json(), token

def download_preview(track_id, preview_url):
    response = requests.get(preview_url)
    if response.status_code == 200:
        file_path = f'previews/{track_id}.mp3'
        with open(file_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Error downloading preview for track ID {track_id}")

def fetch_and_save_album_data(genre, token):
    print(f"Fetching data for genre: {genre}")
    artists_url = f"https://api.spotify.com/v1/search?q=genre:\"{genre}\"&type=artist&limit=50"
    albums_fetched = set()

    artists_response, token = make_request(artists_url, token)
    if not artists_response:
        print(f"Error fetching artists for genre {genre}")
        return

    for artist in artists_response.get('artists', {}).get('items', []):
        artist_id = artist['id']
        artist_name = artist['name']

        albums_url = f"https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album,single&limit=50&market=US"
        albums_response, token = make_request(albums_url, token)
        if not albums_response:
            print(f"Error fetching albums for artist {artist_name} ({artist_id})")
            continue

        for album in albums_response.get('items', []):
            album_id = album['id']
            if album_id in albums_fetched:
                continue
            albums_fetched.add(album_id)

            tracks_url = f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=50&market=US"
            tracks_response, token = make_request(tracks_url, token)
            if not tracks_response:
                print(f"Error fetching tracks for album {album['name']} ({album_id})")
                continue

            album_tracks = []
            for track in tracks_response.get('items', []):
                track_data = {
                    'name': track.get('name'),
                    'id': track.get('id'),
                    'popularity': track.get('popularity'),
                    'duration_ms': track.get('duration_ms'),
                    'preview_url': track.get('preview_url')
                }
                album_tracks.append(track_data)

                preview_url = track.get('preview_url')
                if preview_url:
                    download_preview(track['id'], preview_url)

            album_data = {
                'album_name': album.get('name'),
                'album_id': album_id,
                'release_date': album.get('release_date'),
                'total_tracks': album.get('total_tracks'),
                'popularity': album.get('popularity'),
                'genres': artist.get('genres', []),
                'artist_name': artist_name,
                'artist_id': artist_id,
                'cover_art_url': album.get('images', [{}])[0].get('url'),
                'spotify_url': album.get('external_urls', {}).get('spotify'),
                'tracks': album_tracks
            }
            
            with open(f'data/{album_id}.json', 'w', encoding='utf-8') as f:
                json.dump(album_data, f, ensure_ascii=False, indent=4)

            print(f"Processed album: {album['name']}, Genre: {genre}, Artist: {artist_name}")

for genre in genres:
    try:
        fetch_and_save_album_data(genre, access_token)
    except Exception as e:
        print(f"Error processing genre {genre}: {e}")

Fetching data for genre: pop
Processed album: 1989 (Taylor's Version) [Deluxe], Genre: pop, Artist: Taylor Swift
Processed album: 1989 (Taylor's Version), Genre: pop, Artist: Taylor Swift
Processed album: Speak Now (Taylor's Version), Genre: pop, Artist: Taylor Swift
Processed album: Midnights (The Til Dawn Edition), Genre: pop, Artist: Taylor Swift
Processed album: Midnights (3am Edition), Genre: pop, Artist: Taylor Swift
Processed album: Midnights, Genre: pop, Artist: Taylor Swift
Processed album: Red (Taylor's Version), Genre: pop, Artist: Taylor Swift
Processed album: Fearless (Taylor's Version), Genre: pop, Artist: Taylor Swift
Processed album: evermore (deluxe version), Genre: pop, Artist: Taylor Swift
Processed album: evermore, Genre: pop, Artist: Taylor Swift
Processed album: folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition], Genre: pop, Artist: Taylor Swift
Processed album: folklore (deluxe version), Genre: pop, Artist: Taylor Swift
Processed 

# Download the cover art code:


In [28]:
import os
import json
import requests

folder_path = 'album_cover_art'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

def download_cover_art(album_id, cover_art_url):
    response = requests.get(cover_art_url)
    if response.status_code == 200:
        with open(os.path.join(folder_path, f'{album_id}.jpg'), 'wb') as f:
            f.write(response.content)
            print(f"Downloaded cover art for album {album_id}")
    else:
        print(f"Failed to download cover art for album {album_id}")

data_folder = 'data'
for filename in os.listdir(data_folder):
    if filename.endswith('.json'):
        file_path = os.path.join(data_folder, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            album_id = data['album_id']
            cover_art_url = data['cover_art_url']
            download_cover_art(album_id, cover_art_url)


Downloaded cover art for album 75Q8BLDuURslpcTthbAcsS
Downloaded cover art for album 1IkLYWnY2NNZjpE0z7m3EH
Downloaded cover art for album 28sV7wzuBcHijsmUJJ9kKJ
Downloaded cover art for album 6FIMt58naoGYrOe4Wn2P3n
Downloaded cover art for album 61MgZIlIkgU1Fb4P9MUV2g
Downloaded cover art for album 30R3UJ0v15PDZ9sFupRzdT
Downloaded cover art for album 2vx3BtBMn8Ed3ygEg1OT2t
Downloaded cover art for album 0uGmnnclrvN2ucqgpgpdyz
Downloaded cover art for album 0lgs2Sa82lyX89nBUWyUy6
Downloaded cover art for album 7D2jhStmJWw2u8DhMWnORp
Downloaded cover art for album 246PG3xptmyOv0vke3Aw7Z
Downloaded cover art for album 6qavIPdgOkhfVrlZHPnrMt
Downloaded cover art for album 4rUiYXutkIFPbCr8H8ELoJ
Downloaded cover art for album 7j3QBadmH19j8ucnxTmVIQ
Downloaded cover art for album 1VKFaHoIvkZTzsSRad2jDZ
Downloaded cover art for album 5n42QA68l3zwUMf0vPzGde
Downloaded cover art for album 6N6BvmIc6X6cKhJG4rCaGN
Downloaded cover art for album 2cKDqvsLVZ7NRwUsg1lTrs
Downloaded cover art for alb

# Extract Audio Features for each song

In [16]:


def make_request_with_retry(url, token, retries=3, backoff_factor=0.5):
    """Make a request to the Spotify API with retry logic."""
    headers = {'Authorization': f'Bearer {token}'}
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 401:
                print("Token expired")
                return None
            if response.status_code != 200:
                print(f"Error fetching data: {response.status_code}")
                return None
            return response.json()
        except requests.exceptions.ConnectionError:
            print("Connection error, retrying...")
            time.sleep(backoff_factor * (2 ** attempt))
    print("Failed to fetch data after retries.")
    return None

def fetch_and_save_audio_features(album_id, track_ids, token, output_folder):
    """Fetch audio features for the given track IDs and save to a JSON file."""
    features_for_album = {}
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i+100]
        response = make_request_with_retry(f'https://api.spotify.com/v1/audio-features?ids={",".join(batch_ids)}', token)
        if response:
            for features in response['audio_features']:
                if features:  
                    features_for_album[features['id']] = features

    with open(os.path.join(output_folder, f'{album_id}.json'), 'w') as f:
        json.dump({album_id: features_for_album}, f, indent=4)

def process_albums(folder, token):
    """Process all albums in the specified folder."""
    output_folder = './audio_features'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(folder):
        if filename.endswith('.json'):
            with open(os.path.join(folder, filename), 'r') as f:
                data = json.load(f)
                track_ids = [track['id'] for track in data['tracks']]
                album_id = data['album_id']
                fetch_and_save_audio_features(album_id, track_ids, token, output_folder)

    print("Finished processing albums.")

folder = './data'
access_token = 'BQCMQUpg-ER6f8liln3U-7Rx0W3Sf3IBOZVVaR9HIY_gn9pHIvEK2f33Yj3v6o5LXSNuzTAMD38cg6t-ZaVhlJEfga0a81Kz02VChXIDkj4Zgr4oVh8'  # Replace this with your actual access token
process_albums(folder, access_token)


Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429
Error fetching data: 429


KeyboardInterrupt: 

# Process Missing Albums

In [12]:
import os

data_folder = './data'
audio_features_folder = './audio_features'

data_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(data_folder) if filename.endswith('.json')]
audio_features_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(audio_features_folder) if filename.endswith('.json')]

missing_filenames = [filename for filename in data_filenames if filename not in audio_features_filenames]

missing_filenames

['5n4jQZpHLAWOVccZKttKSD',
 '4UB2nT0IAEunpdUFiuFUxA',
 '6RX23c74RQwa8q1wFSkjPx',
 '1AgLeQdMYoLfUj2FZFuOId',
 '2Scq3cyNzKxmN6L8qtWaCc',
 '43WWZDcWprzKBFL2rqJxPB',
 '6deiaArbeoqp1xPEGdEKp1',
 '6AFJIZmd1zWQDBBwHyeo9w',
 '3jX84c9YrJYvB7nrul5fbN',
 '5SULZpveqjEFxoMoZjudDB',
 '0y9B7AohMcwUhbXl0LrOK0',
 '3KqjipkU3TRvKAsqWTLtVj',
 '3FFGbUutKWN1c4f0CJR4Uh',
 '13qwsJ7ZhrTSPJLN8nhBCH',
 '6E4RZkGvHiOp41XvdpiWaK',
 '4uRfYhBBaSX8N3amojmMBE',
 '5x3ZoZjexntAoipx0KaDnf',
 '7Kvek4EtuzV87BT8C8z3yj',
 '0NpE9IgpJOslyw3TuEJ4b4',
 '72seWTJF9U5SljizfyF2ZK',
 '5S4SuPHbaozi5PDedAONTG',
 '5g2INX6dnMIJG0xKygXwF3',
 '5vgkX6abffhom2ejVI11yq',
 '2USiuZqEAyfOcKmcujg4hs',
 '61eILVGEZUfwMNTCeL4f0S',
 '2EOWRhAyOt6UmBprRs7Ye1',
 '2CIs0r9ZfHh8tpdFM6y9g4',
 '6wwwQnxcyo5Mh94TpZlE5B',
 '4EPQtdq6vvwxuYeQTrwDVY',
 '3uPnO1aZBwMgWK1DI5zve9',
 '6EuKoCk8R63UfuPLa9qr7U',
 '3rrvHWzZCF8TvqT9oJtfVZ',
 '7oQ1JsJdOOf2Rk6DqZPl0B',
 '2ymNS2YXhwg2akYRX1aL6A',
 '5XYiRQmQa5nRFAaUAMWVJ9',
 '1MPAXuTVL2Ej5x0JHiSPq8',
 '7K0mSkUKYnyrOQ4ko2fl7N',
 

In [23]:

def make_request_with_retry(url, token):
    """Make a request to the Spotify API with basic error handling."""
    headers = {'Authorization': f'Bearer {token}'}
    for attempt in range(3):  
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching data: {response.status_code}")
            if response.status_code == 429:  
                print("Error 429")
            else:
                break  
    return None  

def fetch_and_save_audio_features(album_id, track_ids, token, output_folder):
    """Fetch audio features for the given track IDs and save to a JSON file, then wait for 31 seconds."""
    features_for_album = {}
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i+100]
        response = make_request_with_retry(f'https://api.spotify.com/v1/audio-features?ids={",".join(batch_ids)}', token)
        if response and 'audio_features' in response:
            for features in response['audio_features']:
                if features:  
                    features_for_album[features['id']] = features
        else:
            print("Failed to fetch or missing 'audio_features' in response.")
            return

    with open(os.path.join(output_folder, f'{album_id}.json'), 'w') as f:
        json.dump({album_id: features_for_album}, f, indent=4)
    
    print(f"Successfully processed {album_id}. Waiting for 31 seconds to avoid rate limit...")

def process_missing_albums(folder, missing_filenames, token):
    """Process albums listed in missing_filenames."""
    output_folder = './audio_features_new'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for album_id in missing_filenames:
        filename = f'{album_id}.json'
        if os.path.exists(os.path.join(folder, filename)):
            with open(os.path.join(folder, filename), 'r') as f:
                data = json.load(f)
                track_ids = [track['id'] for track in data['tracks']]
                fetch_and_save_audio_features(album_id, track_ids, token, output_folder)
        else:
            print(f"File {filename} does not exist in the data folder.")

    print("Finished processing missing albums.")

import os

data_folder = './data_new'
audio_features_folder = './audio_features_new'

data_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(data_folder) if filename.endswith('.json')]
audio_features_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(audio_features_folder) if filename.endswith('.json')]

missing_filenames = [filename for filename in data_filenames if filename not in audio_features_filenames]


folder = './data_new'
client_id = '9bf29085a2f94ea1a40e718cbc222bb7'
client_secret = 'd90389cc6c1f4cfaa6ec34ee567c0f5c'
access_token = 'BQCtF3tujmRhtK7lv57Syuk85bTnWdTwi-OLEA3fGn9zmcF-mQHS8L3cqMecM5-IKaWCYVDrjaIgc3HScvq0FfQfA1y7MuFel2le2vNW-vKdm9eBjg0'

process_missing_albums(folder, missing_filenames, access_token)

Successfully processed 75Q8BLDuURslpcTthbAcsS. Waiting for 31 seconds to avoid rate limit...
Successfully processed 0uGmnnclrvN2ucqgpgpdyz. Waiting for 31 seconds to avoid rate limit...
Successfully processed 5n42QA68l3zwUMf0vPzGde. Waiting for 31 seconds to avoid rate limit...
Successfully processed 2cKDqvsLVZ7NRwUsg1lTrs. Waiting for 31 seconds to avoid rate limit...
Successfully processed 2XeTY6zdUIUfItgZxXKsiO. Waiting for 31 seconds to avoid rate limit...
Successfully processed 6EtSefhfSVVC4edzfUq2wZ. Waiting for 31 seconds to avoid rate limit...
Successfully processed 1X8VfTtKAhieUbPLF9CtGy. Waiting for 31 seconds to avoid rate limit...
Successfully processed 4xHLjJANf5TEofaEQB19XY. Waiting for 31 seconds to avoid rate limit...
Successfully processed 0SV6jc85NrPS12o3bmieQh. Waiting for 31 seconds to avoid rate limit...
Successfully processed 1jKJhIDQaa1hK5OTLNTc0s. Waiting for 31 seconds to avoid rate limit...
Successfully processed 3IyBnoXfi2MiuIqMlgeKBf. Waiting for 31 seconds 

In [22]:
import os
import requests
import json
import time

access_token = 'BQCtF3tujmRhtK7lv57Syuk85bTnWdTwi-OLEA3fGn9zmcF-mQHS8L3cqMecM5-IKaWCYVDrjaIgc3HScvq0FfQfA1y7MuFel2le2vNW-vKdm9eBjg0'

if not os.path.exists('previews_new'):
    os.makedirs('previews_new')
if not os.path.exists('data_new'):
    os.makedirs('data_new')

def make_request(url):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response

def download_preview(track_id, preview_url):
    response = requests.get(preview_url)
    if response.status_code == 200:
        file_path = f'previews_new/{track_id}.mp3'
        with open(file_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Error downloading preview for track ID {track_id}")

def process_album(album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}"
    response = make_request(url)
    
    if response.status_code == 429:
        print(f"Rate limit exceeded for album {album_id}. Skipping to the next one.")
        return  
    
    if response.status_code == 200:
        album_data = response.json()
        album_tracks = []
        for track in album_data['tracks']['items']:
            track_data = {
                'name': track.get('name'),
                'id': track.get('id'),
                'popularity': track.get('popularity'),
                'duration_ms': track.get('duration_ms'),
                'preview_url': track.get('preview_url')
            }
            album_tracks.append(track_data)

            if track.get('preview_url'):
                download_preview(track['id'], track['preview_url'])

        album_info = {
            'album_name': album_data.get('name'),
            'album_id': album_id,
            'release_date': album_data.get('release_date'),
            'total_tracks': album_data.get('total_tracks'),
            'popularity': album_data.get('popularity'),
            'genres': album_data.get('genres', []),
            'artist_name': album_data['artists'][0].get('name'),
            'artist_id': album_data['artists'][0].get('id'),
            'cover_art_url': album_data['images'][0].get('url') if album_data['images'] else None,
            'spotify_url': album_data['external_urls'].get('spotify'),
            'tracks': album_tracks
        }

        file_name = f"{album_id}.json"  
        with open(f'data_new/{file_name}', 'w', encoding='utf-8') as f:
            json.dump(album_info, f, ensure_ascii=False, indent=4)

        print(f"Processed album: {album_data.get('name')}")
    else:
        print(f"Error fetching album {album_id}: Status code {response.status_code}")


for album_id in missing_album_ids_not_found:
    process_album(album_id)


Processed album: Post Electronic
Processed album: Undertale No AU: Time Paradox (Electronic Version)
Processed album: Daycare Theme (Electronic Mix)
Processed album: Instrumental Electronic Dance Music
Processed album: Electronic Radio Waves
Processed album: Electronic Rhythms
Processed album: Electronic Exploration 1
Processed album: Electronic Machinations
Processed album: Electronic Projects for Musicians
Processed album: Electronic EDM Progressive House Gaming Music Mix
Processed album: Edm Instrumentals: Cool, Trendy, Uplifting Electronic Music
Processed album: Flux
Processed album: Taiko no Tatsujin Namco Original Soundtrack ”Electronic ​Emotion Songs” collection
Processed album: Electronic Sector
Processed album: Dark Tantric Pleasures: Electronic Sensual Beats for Erotic Massage
Processed album: Mrbeast Song Electronic
Processed album: Soul Jazz Records Presents DEUTSCHE ELEKTRONISCHE MUSIK: Experimental German Rock And Electronic Music 1972-83
Processed album: Deep Alpha Focus

KeyboardInterrupt: 

In [26]:
import os
import json
import requests

audio_features_dir = './audio_features_new'
data_dir = './data'
cover_art_dir = './album_cover_art_new'

os.makedirs(cover_art_dir, exist_ok=True)

audio_feature_files = os.listdir(audio_features_dir)

for filename in audio_feature_files:
    data_file_path = os.path.join(data_dir, filename)
    
    try:
        with open(data_file_path, 'r') as file:
            data = json.load(file)
            
            cover_art_url = data.get('cover_art_url')
            if cover_art_url:
                response = requests.get(cover_art_url)
                if response.status_code == 200:
                    cover_art_filename = os.path.basename(cover_art_url)
                    cover_art_path = os.path.join(cover_art_dir, cover_art_filename)
                    with open(cover_art_path, 'wb') as image_file:
                        image_file.write(response.content)
                        print(f"Downloaded and saved: {cover_art_path}")
                else:
                    print(f"Failed to download image from {cover_art_url}")
    except FileNotFoundError:
        print(f"Data file not found for {filename}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {data_file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Downloaded and saved: ./album_cover_art_new/ab67616d0000b273106aa67615b543eff10c3efb
Downloaded and saved: ./album_cover_art_new/ab67616d0000b273e85d2cd11210bdd5c62b570a
Downloaded and saved: ./album_cover_art_new/ab67616d0000b27376c76a63cc5db4c2e1a55861
Downloaded and saved: ./album_cover_art_new/ab67616d0000b273f12512154beeb7a7b64acabd
Downloaded and saved: ./album_cover_art_new/ab67616d0000b2731ffe740dbbff68ec7649289e
Downloaded and saved: ./album_cover_art_new/ab67616d0000b27398602edbca7a76d682c1472b
Downloaded and saved: ./album_cover_art_new/ab67616d0000b273879aead210f3d98c8cc23254
Downloaded and saved: ./album_cover_art_new/ab67616d0000b2738373600eacfc816fe7e419ae
Downloaded and saved: ./album_cover_art_new/ab67616d0000b2737f001a691cf0abaf8ec96832
Downloaded and saved: ./album_cover_art_new/ab67616d0000b273f2af11f171b22cb641d0cd54
Downloaded and saved: ./album_cover_art_new/ab67616d0000b2732e3214175b5fa284ca1107ef
Downloaded and saved: ./album_cover_art_new/ab67616d0000b2738fde0

KeyboardInterrupt: 

# Make sure that the album_id names of json files match

In [36]:
import os

data_dir = 'data'
album_cover_art_dir = 'album_cover_art'

def get_files_in_directory_without_extension(directory_path):
    if os.path.exists(directory_path):
        return [os.path.splitext(file)[0] for file in os.listdir(directory_path)]
    else:
        print(f"The directory '{directory_path}' does not exist.")
        return []

data_files_no_ext = get_files_in_directory_without_extension(data_dir)
album_files_no_ext = get_files_in_directory_without_extension(album_cover_art_dir)

if sorted(data_files_no_ext) == sorted(album_files_no_ext):
    print("\nThe filenames in both directories are identical (ignoring extensions).")
elif set(data_files_no_ext) & set(album_files_no_ext):
    common_files = list(set(data_files_no_ext) & set(album_files_no_ext))
    print("\nCommon filenames in both directories (ignoring extensions):", common_files)
else:
    print("\nThere are no common filenames between the two directories (ignoring extensions).")



The filenames in both directories are identical (ignoring extensions).


In [38]:
data_dir = 'audio_features'
album_cover_art_dir = 'data'

data_files_no_ext = get_files_in_directory_without_extension(data_dir)
album_files_no_ext = get_files_in_directory_without_extension(album_cover_art_dir)

if sorted(data_files_no_ext) == sorted(album_files_no_ext):
    print("\nThe filenames in both directories are identical (ignoring extensions).")
elif set(data_files_no_ext) & set(album_files_no_ext):
    common_files = list(set(data_files_no_ext) & set(album_files_no_ext))
    print("\nCommon filenames in both directories (ignoring extensions):", common_files)
else:
    print("\nThere are no common filenames between the two directories (ignoring extensions).")



The filenames in both directories are identical (ignoring extensions).


# Combine Album+song data with audio features

In [40]:
import json
import os

data_dir = './data'
audio_features_dir = './audio_features'
combined_data_dir = './combined_data'

if not os.path.exists(combined_data_dir):
    os.makedirs(combined_data_dir)

for filename in os.listdir(data_dir):
    if filename.endswith('.json'):
        data_file_path = os.path.join(data_dir, filename)
        audio_features_file_path = os.path.join(audio_features_dir, filename)
        combined_data_file_path = os.path.join(combined_data_dir, filename)
        
        with open(data_file_path, 'r') as file:
            album_data = json.load(file)
        
        if os.path.exists(audio_features_file_path):
            with open(audio_features_file_path, 'r') as file:
                audio_features_data = json.load(file)
                album_audio_features = audio_features_data.get(album_data['album_id'], {})
                
            for track in album_data['tracks']:
                track_id = track['id']
                if track_id in album_audio_features:
                    track.update(album_audio_features[track_id])
                else:
                    print(f"Audio features for track ID {track_id} not found.")
            
            with open(combined_data_file_path, 'w') as file:
                json.dump(album_data, file, indent=4)
            
            print(f"Combined data for album {album_data['album_name']} has been saved.")
        else:
            print(f"Audio features file for {filename} not found.")


Combined data for album The Eternal Rocks Beneath has been saved.
Combined data for album Made In England has been saved.
Combined data for album Freedom. has been saved.
Combined data for album Bam Bam (feat. Ed Sheeran) has been saved.
Combined data for album The Concert has been saved.
Combined data for album I'm Still a Hitta has been saved.
Combined data for album Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy) [Remixes] has been saved.
Combined data for album ON THE ROCKS has been saved.
Combined data for album FERXXOCALIPSIS has been saved.
Combined data for album The Shape I'm Takin' has been saved.
Combined data for album Discoteca (Benny Benassi Remix) has been saved.
Combined data for album Green Light Freemasons EP has been saved.
Combined data for album Rich & Reckless has been saved.
Combined data for album She Knows (with Akon) [The Remixes] has been saved.
Combined data for album Your Heart has been saved.
Combined data for album Relaxing Jazz Music for D

# Since each album is tagged as multiple genres, list all the genres

In [1]:
import os
import json

data_folder_path = 'data/'
genre_counts = {}
data_files = [f for f in os.listdir(data_folder_path) if f.endswith('.json')]
for file_name in data_files:
    file_path = os.path.join(data_folder_path, file_name)
    
    with open(file_path, 'r') as file:
        data = json.load(file)
        
        genres = data.get('genres', [])
        
        for genre in genres:
            if genre in genre_counts:
                genre_counts[genre] += 1
            else:
                genre_counts[genre] = 1

genre_counts

{'glam rock': 100,
 'mellow gold': 50,
 'piano rock': 100,
 'rock': 610,
 'canadian pop': 138,
 'pop': 1543,
 'singer-songwriter pop': 67,
 'uk pop': 112,
 'album rock': 65,
 'classic rock': 143,
 'country rock': 20,
 'southern rock': 20,
 'swamp rock': 20,
 'chicago drill': 94,
 'chicago rap': 144,
 'drill': 94,
 'hip hop': 1043,
 'rap': 1786,
 'trap': 917,
 'emo': 45,
 'modern rock': 331,
 'colombian pop': 89,
 'pop reggaeton': 48,
 'reggaeton': 194,
 'reggaeton colombiano': 48,
 'trap latino': 244,
 'urbano latino': 194,
 'alternative rock': 40,
 'funk metal': 40,
 'funk rock': 40,
 'permanent wave': 205,
 'dance pop': 398,
 'miami hip hop': 131,
 'r&b': 291,
 'atl hip hop': 434,
 'melodic rap': 360,
 'plugg': 58,
 'big room': 49,
 'edm': 144,
 'pop dance': 96,
 'conscious hip hop': 58,
 'north carolina hip hop': 78,
 'pop rap': 574,
 'philly rap': 47,
 'rage rap': 56,
 'detroit hip hop': 45,
 'slap house': 47,
 'k-pop': 45,
 'k-pop boy group': 45,
 'memphis hip hop': 145,
 'souther